In [1]:
import sys
sys.path.append('/source/main')

In [2]:
import os
import logging
from datetime import datetime
import time
from itertools import chain

import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from naruto_skills.new_voc import Voc
from torch.utils.data import DataLoader, Subset, Dataset
import matplotlib.pyplot as plt

from model_def.siamese_core import SiameseModelCore
from model_def.wrap_core_model import WrapSiameseModelCore
from data_for_train.pool import PoolDocs
from utils import pytorch_utils
from preprocess import preprocessor
from data_for_train.index_dataset import IndexDataset
from data_for_train.positive_dataset import PositiveDataset
from naruto_skills.training_checker import TrainingChecker

In [3]:
logging.basicConfig(level=logging.INFO)
pd.set_option('display.max_colwidth', -1)

In [7]:
def docs2input_tensors(docs, device):
    preprocessed_docs = [preprocessor.infer_preprocess(doc) for doc in docs]
#     max_len = max([len(item.split()) for item in preprocessed_docs])
    max_len = 100
    preprocessed_docs = [' '.join(doc.split()[:max_len]) for doc in preprocessed_docs]
    word_input = voc.docs2idx(preprocessed_docs, equal_length=max_len)
    inputs = np.array(word_input)
    input_tensors = torch.from_numpy(inputs)
    input_tensors = input_tensors.to(device)
    return input_tensors

def predict_batch(docs):
    with torch.no_grad():
        input_tensors = docs2input_tensors(docs, device)
        predict_tensor = model(input_tensors)
        predict_np = predict_tensor.cpu().numpy()
        return predict_np[:, 1]

def predict_docs(docs, batch_size):
    return list(chain(*[predict_batch(docs[i: i+batch_size]) for i in tqdm(range(0, len(docs), batch_size))]))

# 1. Data loading

## 1.1 Pool

In [8]:
def collate_fn(list_data):
    """
    shape == (batch_size, col1, col2, ...)
    """
    data = zip(*list_data)
    data = [np.stack(col, axis=0) for col in data]
    data = [torch.from_numpy(col) for col in data]
    return data
voc = Voc.load('/source/main/vocab/output/voc.pkl')
MAX_LENGTH = 100
BATCH_SIZE = 256
EXP_ID = '13'

In [9]:
# df_neg = pd.read_csv('/source/main/data_for_train/output/huge_pool/wiki.csv', nrows=1e6, usecols=['target'])
# df_neg.rename(columns={'target': 'mention'}, inplace=True)
# df_neg.dropna(inplace=True, subset=['mention'])
# df_neg.drop_duplicates(inplace=True, subset=['mention'])
# df_neg = df_neg.iloc[:794323, ]

In [10]:
# print(df_neg.shape)
# neg = IndexDataset(voc, list(df_neg['mention']), equal_length=MAX_LENGTH)
# neg = PoolDocs(neg)

In [11]:
df_pool = pd.read_csv('/source/main/data_for_train/output/train/pool.csv', nrows=1e6)
df_pool.dropna(inplace=True, subset=['mention'])
df_pool.drop_duplicates(inplace=True, subset=['mention'])
df_pool = df_pool.iloc[:794323, :]

In [12]:
print(df_pool.shape)
pool = IndexDataset(voc, list(df_pool['mention']), equal_length=MAX_LENGTH)
pool = PoolDocs(pool)

(794323, 2)


INFO:root:Total: 794323


In [13]:
len(pool)

794323

## 1.2 Positive data

In [14]:
df_pos = pd.read_csv('/source/main/data_for_train/output/train/positive_class_1.csv')
df_pos.dropna(inplace=True, subset=['mention'])
df_pos.drop_duplicates(inplace=True, subset=['mention'])

print(df_pos.shape)
pos = IndexDataset(voc, list(df_pos['mention']), equal_length=MAX_LENGTH)
pos = PositiveDataset(pos)

(2494, 3)


In [ ]:
pos[0][:10]

In [ ]:
# pool_data, _ = zip(*pool)
# pool_data = [str(item) for item in pool_data]
# pos_data, _ = zip(*pos)
# pos_data = [str(item) for item in pos_data]
# len(set(pool_data).intersection(set(pos_data)))

In [ ]:
# df_anchor = pd.read_csv('/source/main/data_for_train/output/train/positive_class_1.csv')
# df_anchor.dropna(inplace=True, subset=['mention'])
# df_anchor.drop_duplicates(inplace=True, subset=['mention'])
# df_anchor = df_anchor.sample(df_anchor.shape[0], random_state=43)
# print(df_anchor.shape)
# anchor = IndexDataset(voc, list(df_anchor['mention']), equal_length=MAX_LENGTH)
# # anchor = PositiveDataset(anchor)

In [15]:
data_loader = DataLoader(dataset=pos+pool, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=collate_fn)

In [16]:
data = next(iter(data_loader))

In [17]:
voc.idx2docs(data[0].cpu().numpy()[:3, :])

['cháu tiếp __o__',
 'ôi xời may mà xem lại nha cm , hai cái đều là thật trog bịch bỉm có cái nào __o__ hệt cái nào đâu , xem thấy bài vội dậy lục xem luôn',
 'ghê thiệt tiết kiệm và giản dị là sang , còn đi mua đồ hiệu là kém sang và sống kì : ) ) ) đúng là bạn rmit kia k đúng thật , nhưng cá nhân mình thấy có lẽ vì khác biệt lối sống nên các bạn bị bạn ấy cách biệt thôi . mình chọn lối sống kém sang của bạn kia đó , sống sướng hơn lối sống sang của bạn nhiều bạn ạ']

In [18]:
core_model = SiameseModelCore(voc.get_embedding_weights())
model = WrapSiameseModelCore(core_model)

In [19]:
model.train()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [20]:
pytorch_utils.count_parameters(model)

13886932

In [21]:
def train_step(inputs):
    model.train()
    step_loss = model.train_batch(inputs[0], inputs[1])
    return step_loss

In [22]:
len(data_loader)

3113

In [23]:
# model.eval()
# # import pdb; pdb.set_trace()
# predict_docs(['giá bao tiền', 'ee', 'Giảm giá sốc'], batch_size=2)

In [ ]:
anchor = docs2input_tensors(['giá bao tiền'], device)
pos = docs2input_tensors(['bao nhiêu thế ?'], device)
neg = docs2input_tensors(['hôm nay tôi đi học'], device)

In [ ]:
print('dis_1', model.get_distance(anchor, pos))
print('dis_2', model.get_distance(anchor, neg))

# 2. Training

## 2.1 Epoch 1

In [24]:
# model.build_stuff_for_training(device)
for epoch_idx in range(1):
    start = time.time()
    for idx, inputs in tqdm(enumerate(data_loader)):
        inputs = [i.to(device) for i in inputs]
        l = train_step(inputs)
        if idx % 100 == 0:
            logging.info('\t Step: %s Loss: %.5f Pos/Neg: %s/%s', idx, l, inputs[1].sum().cpu().item(), 
                         inputs[1].size(0))
    duration = time.time() - start
    logging.info('Epoch %s took %.2f s', epoch_idx, duration)

0it [00:00, ?it/s]INFO:root:	 Step: 0 Loss: 0.56506 Pos/Neg: 1/256
100it [00:24,  4.08it/s]INFO:root:	 Step: 100 Loss: 0.00714 Pos/Neg: 1/256
200it [00:49,  4.06it/s]INFO:root:	 Step: 200 Loss: 0.02240 Pos/Neg: 2/256
300it [01:13,  4.04it/s]INFO:root:	 Step: 300 Loss: 0.01137 Pos/Neg: 1/256
400it [01:38,  4.04it/s]INFO:root:	 Step: 400 Loss: 0.00009 Pos/Neg: 0/256
500it [02:03,  4.05it/s]INFO:root:	 Step: 500 Loss: 0.01345 Pos/Neg: 1/256
600it [02:28,  4.06it/s]INFO:root:	 Step: 600 Loss: 0.00848 Pos/Neg: 1/256
700it [02:52,  4.06it/s]INFO:root:	 Step: 700 Loss: 0.00885 Pos/Neg: 1/256
800it [03:17,  4.06it/s]INFO:root:	 Step: 800 Loss: 0.00533 Pos/Neg: 0/256
900it [03:42,  4.04it/s]INFO:root:	 Step: 900 Loss: 0.00669 Pos/Neg: 0/256
1000it [04:06,  4.04it/s]INFO:root:	 Step: 1000 Loss: 0.00873 Pos/Neg: 1/256
1100it [04:31,  4.04it/s]INFO:root:	 Step: 1100 Loss: 0.00240 Pos/Neg: 1/256
1200it [04:56,  4.06it/s]INFO:root:	 Step: 1200 Loss: 0.00136 Pos/Neg: 0/256
1300it [05:21,  4.05it/s]IN

In [ ]:
# for epoch_idx in range(10):
#     start = time.time()
#     model.train()
#     for idx, inputs in tqdm(enumerate(data_loader)):    
#         inputs = [i.to(device) for i in inputs]
#         l = train_step(inputs)
#     duration = time.time() - start
#     logging.info('Epoch %s took %.2f s', epoch_idx, duration)
    
#     model.eval()    
#     df_pos['pred'] = predict_docs(df_pos['mention'], batch_size=256)
#     df_pool['pred'] = predict_docs(df_pool['mention'], batch_size=256)
    
#     logging.info('Recall: %s/%s=%.4f', (df_pos['pred']>=0.5).sum(), df_pos.shape[0], 
#                  (df_pos['pred']>=0.5).sum()/df_pos.shape[0])
#     logging.info('Ratio on pool: %s/%s=%.4f', (df_pool['pred']>=0.5).sum(), df_pool.shape[0], 
#                  (df_pool['pred']>=0.5).sum()/df_pool.shape[0])
    

In [ ]:
# fig = plt.figure(figsize=(10, 5))

# ax = fig.add_subplot(1, 2, 1)
# df_pos.loc[:500, 'pred'].hist(bins=100, ax=ax)
# ax.set_title('Spy')
# ax.set_xlim(0, 0.9)
# ax.set_ylim(0, 100)

# ax = fig.add_subplot(1, 2, 2)
# df_pos.loc[500:, 'pred'].hist(bins=100, ax=ax)
# ax.set_title('Positive')
# ax.set_xlim(0, 0.9)
# ax.set_ylim(0, 100)

# plt.show()


In [ ]:
# df_pool_social = pd.read_csv('/source/main/data_for_train/output/train/pool.csv')
# df_pool_social.rename(columns={'target': 'mention'}, inplace=True)
# df_pool_social.dropna(inplace=True, subset=['mention'])
# df_pool_social.drop_duplicates(inplace=True, subset=['mention'])
# df_pool_social = df_pool_social.iloc[:794323, ]

In [25]:
EXP_ID='13b'

In [26]:
training_checker = TrainingChecker(model, root_dir='/source/main/train/output/saved_models/%s/' % EXP_ID,
                                   init_score=-10000)
training_checker.save_model()

INFO:root:Model will be saved at /source/main/train/output/saved_models/13b/


'/source/main/train/output/saved_models/13b/None.pt'

# Analyse

In [27]:
model.eval()

In [28]:
df_pos['mention'].sample(3)

2404    __d__ ink __d__k co gia bao nhieu vay shop       
1812    có màn hình cong __d__ ” k ạh shop ? giá bnhiu ạh
950     friso dah cho be tu __d__t nhju __d__hop ạ      
Name: mention, dtype: object

In [30]:
model.eval()
predict_docs(['giá bao tiền', 'ee', 'Giảm giá sốc'], batch_size=1)

100%|██████████| 3/3 [00:00<00:00, 111.80it/s]


[0.36913168, 4.776274e-05, 0.009863002]

In [31]:
import ast

import pandas as pd
pd.set_option('display.max_colwidth', -1)
from sklearn import metrics

from data_for_train.index_dataset import IndexDataset
from data_for_train.positive_dataset import PositiveDataset
from data_for_train import pool
from naruto_skills.new_voc import Voc

## Recall

### Eval

In [32]:
df_pos_eval = pd.read_csv('/source/main/data_for_train/output/eval/positive_class_1.csv')
df_pos_eval = df_pos_eval.drop_duplicates(subset=['mention'])

In [33]:
df_pos_eval['pred'] = predict_docs(list(df_pos_eval['mention']), batch_size=256)
# print(sum(df_pos_eval['pred']>=0.5)/df_pos_eval.shape[0])
# print(df_pos_eval.shape)

100%|██████████| 2/2 [00:00<00:00, 15.29it/s]


In [34]:
(df_pos_eval['pred']>=0.5).sum()/df_pos_eval.shape[0]

0.47335423197492166

### Test

In [36]:
df_pos_test = pd.read_csv('/source/main/data_for_train/output/test/positive_class_1.csv')
df_pos_test = df_pos_test.drop_duplicates(subset=['mention'])

In [37]:
df_pos_test['pred'] = predict_docs(list(df_pos_test['mention']), batch_size=256)
print(sum(df_pos_test['pred']>=0.5)/df_pos_test.shape[0])
print(df_pos_test.shape)

100%|██████████| 2/2 [00:00<00:00,  9.38it/s]

0.4454828660436137
(321, 4)


## Score: pr/P(y=1)

### Eval

In [38]:
df_pool_eval = pd.read_csv('/source/main/data_for_train/output/eval/pool.csv')

In [39]:
df_pool_eval['pred'] = predict_docs(list(df_pool_eval['mention']), batch_size=256)

# print(sum(df_pool_eval['pred']>=0.5)/df_pool_eval.shape[0])
# print(df_pool_eval.shape)

100%|██████████| 391/391 [00:41<00:00,  9.50it/s]


In [40]:
sum(df_pool_eval['pred']>=0.5)/df_pool_eval.shape[0]

0.01988

In [41]:
df_pool_eval[df_pool_eval['pred']>=0.5].sample(5)

,mention_type,mention,pred
65902,1,< div class= '' meta '' > < h__d__ class= '' heading cms-title '' > giá vàng hôm nay ngày __d__/__d__ : vàng giữ đà tăng ngày đầu tuần < /h__d__ > < div id= '' ctl__d___maincontent_plargeimage '' class= '' photo cms-body '' > < img class= '' cms-photo '' src= '' https : //image.tinnhanhchungkhoan.vn/w__d__/uploaded/__d__/bpikpjik/__d_____d_____d__/gia-vang-hom-nay-ngay-__d__-__d___rver.jpg '' alt= '' giá vàng hôm nay ngày __d__/__d__ : vàng giữ đà tăng ngày đầu tuần '' width= '' __d__ '' > < p > < /p > < p class= '' imgdesc '' > ảnh shutterstock. < /p > < /div > <,0.999968
41119,1,"< h__d__ class= '' article-content__d__ article-content__d__ '' > < div id= '' ctl__d___placeholdermain_g_d__d__e__d__f__d_____d__e__d_____d__d__d_____d__fab___d__aead__d__db__d__ '' > ( tbtco ) - thị trường giao dịch đầy bất ngờ trong ngày hôm nay ( __d__/__d__ ) khi xuất hiện một đợt xả hàng mạnh , đẩy đại đa số cổ phiếu giảm giá . vn-index bị ép mạnh giảm __d__,__d__ điểm , về sát __d__ điểm . < /div > < /h__d__ > < div id= '' ctl__d___placeholdermain_g___d__f__d__c__d_____d_____d___a__d__f__d___c__d__bc__d__ '' > < /div > < div class= '' article-content article-content__d__ '' > < div id= '' ctl__d___placeholdermain_g_ab__d__e__d_____d__f__d_____d__a__d___a__d__bf___d__b__d__c__d__a__d__ '' > < p > < strong",0.978239
6387,2,"< div class= '' bbcodeblock bbcodequote '' > < div class= '' attribution type '' > thanglong__d__ đã viết : < a class= '' attributionlink '' > ↑ < /a > < /div > < blockquote class= '' quotecontainer '' > < div class= '' quote '' > nguyên văn cafef thế này `` rủi ro tăng dần , hồi phục là cơ hội giãm tỉ trọng cổ phiếu `` . hi , tát nước theo mưa thì dễ , biết khi nào sắp mưa , để mình tát nước ao trước chuẩn bị , giờ ngồi chơi ngắm mưa",0.999707
16740,2,< span > < b > toàn cảnh nợ xấu tại __d__ ngân hàng trong __d__ tháng đầu năm < /b > < /span > < br > __d__-__d__-__d__ - __d__:__d__ pm | < a class= '' externallink '' rel= '' nofollow '' > tài chính - ngân hàng < /a > < br > < br > < a class= '' externallink '' rel= '' nofollow '' > < /a > < br > < img src= '' http : //cafefcdn.com/thumb_w/__d__/__d__/__d__/__d__/photo__d__-__d__-__d__-crop-__d__.jpg '' class= '' bbcodeimage lbimage '' alt= '' [ ​img ] '' > < br > < span,0.999985
63483,1,"< iframe frameborder= '' __d__ '' marginheight= '' __d__ '' marginwidth= '' __d__ '' width= '' __d__ % '' id= '' detail_iframe_id '' > < /iframe > < br > samsung vừa âm thầm ra mắt hai sản phẩm mới thuộc phân khúc trung cấp , galaxy j__d__+ và j__d__+ trên trang chủ của hãng. < br > với galaxy j__d__+ , sản phẩm có màn hình __d__ inch hd , chip lõi tứ tốc độ __d__,__d__ ghz , với loại chip sử dụng sẽ tùy thuộc vào từng thị trường cụ thể ( dự kiến là snapdragon __d__ ở",0.982572


### Test

In [55]:
df_pool_test = pd.read_csv('/source/main/data_for_train/output/test/pool.csv')

In [56]:
df_pool_test.head()

,mention_type,mention
0,2,thao anh pham
1,2,vũ đức hoàng
2,2,ha nhy chị nè
3,2,hoàng thị thu trang m ở thái bình m nhé.m đặt qua shoppe hoặc sendo sẽ dc freeship __d__k___d__k nha
4,2,minh minh đi ăn cho bằng hết nhá


In [57]:
df_pool_test['pred'] = predict_docs(list(df_pool_test['mention']), batch_size=256)

100%|██████████| 391/391 [00:41<00:00,  9.53it/s]


In [58]:
print(sum(df_pool_test['pred']>=0.5)/df_pool_test.shape[0])
print(df_pool_test.shape)

0.02103
(100000, 3)


In [45]:
df_pool_eval[df_pool_eval['pred']>=0.5].sample(200).to_csv('13b.csv', index=None)

In [113]:
predict_docs(batch_size=1, docs=[""" < img src= '' http : //cafefcdn.com/zoom/__d_____d__/__d__/__d__/__d__/photo__d__-__d__.jpg '' title= '' bvh tăng trần “ trắng bên bán ” , vnindex tiếp tục tăng hơn __d__ điểm '' > < br > tạm dừng phiên giao dịch buổi sáng , vnindex tăng __d__,__d__ điểm ( __d__,__d__ % ) lên __d__.__d__,__d__ điểm ; hnx-index tăng __d__,__d__ điểm ( __d__,__d__ % ) lên __d__,__d__ điểm ; upcom-index tăng __d__,__d__ điểm ( __d__,__d__ % ) lên __d__,__d__ điểm . < br > < span id= '' maincontent '' > < p > càng về cuối buổi sáng , giao dịch càng trở nên tích """])

100%|██████████| 1/1 [00:00<00:00, 10.97it/s]


[0.65487015]

In [109]:
df_pool_eval[df_pool_eval['mention']=="""có nikon d__d__ ko bạn ? cho xin cái giá < div class= '' messagetextendmarker '' > < /div >"""]

,mention_type,mention,pred
28207,2,có nikon d__d__ ko bạn ? cho xin cái giá < div class= '' messagetextendmarker '' > < /div >,0.97319


In [78]:
df_pool_eval[df_pool_eval['pred']>=0.5].sample(10)

,mention_type,mention,pred
14268,2,"< a class= '' username author bdposttree_replyposter '' dir= '' auto '' > vtb center < /a > < div class= '' bdposttree_parentquote hash-b__d__ea__d__b__d__e__d__e__d__c__d__a__d__f__d__aee '' > < div class= '' bbcodeblock bbcodequote '' > < div class= '' attribution type '' > conchott đã nói : < a class= '' attributionlink '' > ↑ < /a > < /div > < blockquote class= '' quotecontainer '' > < div class= '' quote '' > chơi dữ thật , .chắc thấy apple bị nv nên cũng sợ < /div > < div class= '' quoteexpand '' > bấm để mở rộng",1.000000
35910,1,< h__d__ > < img src= '' http : //nguoilambao.vn/templates/default/images/logo.png '' alt= '' vpbank lọt top __d__ doanh nghiệp tư nhân lớn nhất việt nam __d__ '' > - ngân hàng tmcp việt nam thịnh vượng ( vpbank ) vừa được vnr đánh giá nằm trong top __d__ doanh nghiệp tư nhân lớn nhất và top __d__ doanh nghiệp tư nhân có lợi nhuận tốt nhất việt nam . < /h__d__ > < br > < p > < img alt= '' '' src= '' http : //nguoilambao.vn/upload_images/images/vpbank_web.jpg '' > < /p > < p > < em > vpbank,0.921351
928,1,"trong khi nhiều ngân hàng tích cực đưa nợ xấu về dưới __d__ % , một số trường hợp có nợ xấu cao do đi vào các lĩnh vực “ mạo hiểm ” . < br > < center > < table class= '' paper-box node '' id= '' __d__ '' align= '' center '' > < tbody > < tr > < td > < img class= '' paper-img '' alt= '' '' src= '' http : //cdn.baogiaothong.vn/files/baogiay/__d__/__d__/__d__/__d__-__d__.jpg '' > < /td > < /tr > < tr > < td > < p class= '' caption '' align=",0.999936
36530,2,"bác nào rành thủ tục trả góp cho mình hỏi , mình đang tính mua __d__ cái body sony a__d__ iii giá niêm yết là __d__tr và muốn trả góp , mình đang dùng thẻ credit của hdbank thì trả góp thế nào là kinh tế nhất ạ ? phí chuyển đổi thẻ là phí gì vậy các bác ? < br > < br > mình cảm ơn < img src= '' http : //forums.voz.vn/images/smilies/off/big_smile.gif '' border= '' __d__ '' alt= '' '' title= '' big smile '' class= '' inlineimg '' >",0.666543
40366,2,mời các cụ luận sấm trạng trình < br > < br > mười phần chết bảy còn ba < br > chết hai còn một mới ra thái bình . < div class= '' messagetextendmarker '' > < /div >,0.810457
54178,1,"< p class= '' leada '' > vietcombank rao bán tới __d__,__d__ triệu cổ phiếu mbb , nhưng chỉ có __d__ nhà đầu tư đăng ký mua tổng cộng __d__,__d__ triệu cổ phiếu . tuy nhiên , giá trị giao dịch của mbb trong ngày __d__/__d__ vẫn đạt __d__.__d__ đồng/cp , cao hơn __d__ % so với giá trị chào bán. < /p > < div > < img border= '' __d__ '' src= '' http : //www.xaluan.com/images/news/image/__d__/__d__/__d__/__d__bbd__d__aab__d__c__d__.img.jpg '' alt= '' vietcombank chỉ bán được __d__ % tổng số cổ phần mbbank chào bán '' class= '' headimg '' >",0.651308
23195,1,"< span > bizlive - < /span > sau daiabank , pgbank sẽ sáp nhập vào hdbank , điều này sẽ giúp hdbank rút ngắn thời gian để trở thành một trong __d__ ngân hàng lớn nhất việt nam . < div class= '' social '' > < div > < a class= '' zshare-zalo zalo-share-button '' > < span class= '' zshare-label '' > chia sẻ < /span > < /a > < div class= '' fb-send '' > < /div > < div class= '' fb-like '' > < /div > < /div > < div > <",0.636466
17149,1,"< div class= '' sapo_detail '' > < p > < span > thcl < /span > - sau ngân hàng tmcp đầu tư và phát triển việt nam ( bidv ) tăng phí dịch vụ thì gần đây các nhiều ngân hàng cũng đồng loạt thông báo điều chỉnh phí giao dịch . theo đó , các ngân hàng giải thích việc tăng phí là để bù đắp các chi phí đầu tư. < /p > < /div > < div id= '' cotent_detail '' class= '' pkg '' > < p > ngân hàng tmcp đầu tư và",0.640712
90801,1,< div class= '' item_slide_show clearfix '' > < div class= '' block_thumb_slide_show '' > < img alt= '' __d__ tủ lạnh tốt nhất __d__ '' class= '' left displayafterresize vne_lazy_image '' > < a class= '' icon_thumb_videophoto icon_thumb_zoom btn_icon_show_slide_show '' > < i class= '' ic ic-expand '' > < /i > < /a > < /div > < div class= '' desc_cation '' > < p > < strong > panasonic nr-dz__d

In [71]:
x=['< div class= '' bbcodeblock bbcodequote '' > < div class= '' attribution type '' > quyet__d__ đã viết : < a class= '' attributionlink '' > ↑ < /a > < /div > < blockquote class= '' quotecontainer '' > < div class= '' quote '' > '' trước đó , sacombank đã liên tiếp đạt các giải thưởng quan trọng như giải < b > ngân hàng có mạng lưới chấp nhận thẻ hiệu quả nhất __d__ < /b > do hiệp hội các ngân hàng việt nam phối hợp với tập đoàn dữ liệu quốc tế idg']

In [73]:
x

['< div class=  bbcodeblock bbcodequote  > < div class=  attribution type  > quyet__d__ đã viết : < a class=  attributionlink  > ↑ < /a > < /div > < blockquote class=  quotecontainer  > < div class=  quote  >  trước đó , sacombank đã liên tiếp đạt các giải thưởng quan trọng như giải < b > ngân hàng có mạng lưới chấp nhận thẻ hiệu quả nhất __d__ < /b > do hiệp hội các ngân hàng việt nam phối hợp với tập đoàn dữ liệu quốc tế idg']

In [74]:
predict_docs(x, batch_size=2)

100%|██████████| 1/1 [00:00<00:00, 25.11it/s]


[2.0627836e-11]

## Precision

In [ ]:
voc = Voc.load('/source/main/vocab/output/voc.pkl')
positive_data = PositiveDataset('/source/main/data_for_train/output/positive_class_1.csv')

In [ ]:
df_pos = pd.DataFrame({'mention': list(positive_data)})


In [ ]:
df_pos['pred'] = predict_docs(df_pos['mention'], batch_size=256)

In [ ]:
df_pos['pred'].iloc[:100].describe()

In [ ]:
df_pos['pred'].describe()

In [ ]:
print((df_pos['pred'] > 0.5).sum()/df_pos.shape[0])

In [ ]:
negative_data = PoolDocs('/source/main/data_for_train/output/huge_pool/')
df_pool = pd.DataFrame({'mention': list(negative_data)})

In [ ]:
df_pool['pred'] = predict_docs(df_pool['mention'], batch_size=256)

In [ ]:
(df_pool['pred']>0.5).sum()

In [ ]:
df_pool[df_pool['pred']>0.5]

In [ ]:
5406/200000*512

In [ ]:
predict_batch(['cái đó bao nhiu '])

In [ ]:
df_pool[(df_pool['pred']>0.5)].sample(10)

## Precision on preserved topics

In [ ]:
df_tags = pd.read_csv('/source/main/data_download/output/auxiliary/app_tags.csv')
df_tags = df_tags[['id', 'name']]
df_tags_price = df_tags[df_tags['name'].map(lambda x: x.lower().find('price neutral') != -1)]
tags_price = set(df_tags_price['id'])

In [ ]:
df_topics = pd.read_csv('/source/main/data_download//output/from_sentiment_price/data_for_evaluate.csv')
df_topics.dropna(subset=['id', 'topic_id', 'sentiment', 'sentiment_auto', 'tags', 'search_text'], inplace=True, how='any')
df_topics['search_text'] = df_topics['search_text'].map(ast.literal_eval)
df_topics['mention'] = df_topics['search_text'].map(lambda x: x[1])
df_topics = df_topics[['id', 'topic_id', 'sentiment', 'sentiment_auto', 'tags', 'mention']]

In [ ]:
df_topics['tags'] = df_topics['tags'].map(ast.literal_eval)

In [ ]:
df_topics['price_neutral'] = df_topics['tags'].map(lambda x: not tags_price.isdisjoint(x))
df_topics['price_neutral'].sum()

In [ ]:
df_topics.shape

In [ ]:
docs = list(df_topics['mention'].map(lambda x: ' '.join(x.split()[:150])))

In [ ]:
df_topics['pred'] = predict_docs(docs, batch_size=128)

In [ ]:
print((df_topics['pred']>0.5).sum())

In [ ]:
print(metrics.classification_report(y_true=df_topics['price_neutral'], y_pred=df_topics['pred']>=0.5))

In [ ]:
df_topics[df_topics['pred']>=0.5].sample(10)

In [ ]:
df_topics[(df_topics['pred']<0.5) & (df_topics['mention'].map(lambda x: x.find('giá') != -1))].sample(10)

In [ ]:
device = torch.device('cuda')
checkpoint = torch.load('/source/main/train/output/saved_models/temp/None.pt', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [ ]:
predict_docs(['giá bao tiền', 'ee'], batch_size=1)